In [1]:
import os
import glob
import csv

import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

from keras.applications import mobilenet
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import layers, models
import numpy as np
import tensorflow as tf

from csv import reader

# libraries for outputting camera images
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
# 'data/train_cv_224/training_labels_handOverFace_n424.csv'
csv_filename = 'data/train_cv_224/training_labels_handOverFace_n424_5cats.csv'
targetDir = 'data/train_cv_224'

train_data = pd.read_csv(csv_filename, sep=',', header=0)

# settuping the kfolds
kf = KFold(n_splits=10, shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [3]:
idg = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input)

In [ ]:
img_size=224
columns = ['0','1','2','3','4']

def create_new_model():
    base_model = mobilenet.MobileNet(weights='imagenet', 
                                     include_top=False, 
                                     input_shape=(img_size, img_size, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(len(columns), activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='mobileNet_n871_5cats-{epoch:02d}-{val_loss:.2f}.h5', 
    verbose=1, monitor='val_accuracy', save_best_only=True
)

In [ ]:
%%notify
### FOR CROSS VALIDATION
## running only one of it
valid_accuracy = []
valid_loss = []
batch_size=32
num_epochs=10
num_samples = train_data.shape[0]

for train_index, val_index in list(kf.split(np.zeros(num_samples), train_data)):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    # one hot encoding because it is softmax    
    train_data_generator = idg.flow_from_dataframe(dataframe=training_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size, 
                                                  class_mode='raw')
    valid_data_generator = idg.flow_from_dataframe(dataframe=validation_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size,
                                                  class_mode='raw')
    
    model = create_new_model()
    
    step_size = train_data_generator.n//train_data_generator.batch_size
    step_size_valid = valid_data_generator.n//valid_data_generator.batch_size

    with tf.device('/device:GPU:0'):
        history = model.fit(train_data_generator, epochs=num_epochs, steps_per_epoch=step_size, 
                        validation_data=valid_data_generator,
                       validation_steps=step_size_valid,
                        callbacks=[])
    
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    valid_accuracy.append(results['accuracy'])
    valid_loss.append(results['loss'])

In [ ]:
### FOR GETTING A GOOD MODEL
### FOR CROSS VALIDATION
%%notify
## running only one of it
valid_accuracy = []
valid_loss = []
batch_size=32
num_epochs=20
num_samples = train_data.shape[0]

for train_index, val_index in list(kf.split(np.zeros(num_samples), train_data)):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    # one hot encoding because it is softmax    
    train_data_generator = idg.flow_from_dataframe(dataframe=training_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size, 
                                                  class_mode='raw')
    valid_data_generator = idg.flow_from_dataframe(dataframe=validation_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size,
                                                  class_mode='raw')
    
    model = create_new_model()
    
    step_size = train_data_generator.n//train_data_generator.batch_size
    step_size_valid = valid_data_generator.n//valid_data_generator.batch_size

    with tf.device('/device:GPU:0'):
        history = model.fit(train_data_generator, epochs=num_epochs, steps_per_epoch=step_size, 
                        validation_data=valid_data_generator,
                       validation_steps=step_size_valid,
                        callbacks=[model_checkpoint_callback])
    
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    valid_accuracy.append(results['accuracy'])
    valid_loss.append(results['loss'])
    break

In [ ]:
model.save('mobileNet_n871_5cats-0.83.h5')

In [ ]:
with open('trainHistoryDict_mobileNet_n871_6cats', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
import pickle
with open("valid_loss_accuracy_k10_n424_cats6.pickle", "wb") as handle:
    pickle.dump([valid_loss, valid_accuracy], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("valid_loss_accuracy_k10_n871_cats6.pickle", "rb") as handle:
    [valid_loss, valid_accuracy] = pickle.load(handle)

In [ ]:
np.mean(valid_accuracy)

In [ ]:
model.save('k10_n424_cats6_v1.h5', save_format='h5')

In [ ]:
predict_labels = {
    0: 'Nobody',
    1: 'Normal sitting',
    2: 'Nose Picking',
    3: 'Biting Nails',
    4: 'Touching eyes',
    5: 'Touching the rest of face'
}

#Load the saved model
model = models.load_model('k10_n871_cats6_v1.h5')
video = cv2.VideoCapture(0)

while True: 
    _, frame = video.read()
    RGB_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(RGB_img)
    
    im = im.resize((224, 224))
    img_array = np.array(im)
    img_array = np.expand_dims(img_array, axis=0)
    print(img_array.shape)
    img_array = mobilenet.preprocess_input(img_array)
    print(img_array.shape)
    
    predictions = model.predict(img_array)
    prediction = predict_labels[np.argmax(predictions)]
    plt.imshow(RGB_img)
    plt.show()
    print(prediction)
    clear_output(wait=True)

video.release()
cv2.destroyAllWindows()

In [ ]:
models.load_model('mobileNet_n871_6cats')